# Customize and Access NSIDC DAAC Data

This notebook will walk you through how to programmatically access data from the NASA National Snow and Ice Data Center Distributed Active Archive Center (NSIDC DAAC) using spatial and temporal filters, as well as how to request customization services including subsetting, reformatting, and reprojection. No Python experience is necessary; each code cell will prompt you with the information needed to configure your data request. The notebook will print the resulting API command that can be used in a command line, browser, or in Python as executed below.

### Import packages


In [1]:
import requests
import json
import zipfile
import io
import math
import os
import glob
import shutil
import pprint
import re
import time
from statistics import mean
%matplotlib inline

short_name = 'SPL4SMGP'
out_path = r'..\1_data'

if not os.path.exists(out_path):
   os.makedirs(out_path)

if not os.path.exists(os.path.join(out_path, short_name)):
   os.makedirs(os.path.join(out_path, short_name))

### Input Earthdata Login credentials

An Earthdata Login account is required to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register.

In [2]:
my_credential_path = "./auth.json"
with open(my_credential_path, 'r') as infile:
    my_credentials = json.load(infile)
    
uid = my_credentials['username'] # Enter Earthdata Login user name
pswd = my_credentials['password'] # Enter Earthdata Login password
email = my_credentials['email'] # Enter Earthdata login email 

### Select data set and determine version number

Data sets are selected by data set IDs (e.g. MOD10A1), whic are also referred to as a "short name". These short names are located at the top of each NSIDC data set landing page in gray above the full title.

In [3]:
# Get json response from CMR collection metadata

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)

The most recent version of  SPL4SMGP  is  007


### Select time period of interest

In [4]:
#Input temporal range 

# start_date = '2015-03-31'# input('Input start date in yyyy-MM-dd format: ')
# start_date = '2015-12-05'# input('Input start date in yyyy-MM-dd format: ')
# start_date = '2016-05-25'# input('Input start date in yyyy-MM-dd format: ')
# 2021-10-01
start_date = '2018-07-01'# input('Input start date in yyyy-MM-dd format: ')
start_time = '00:00:00' # input('Input start time in HH:mm:ss format: ')
end_date = '2019-01-01' # input('Input end date in yyyy-MM-dd format: ')
end_time = '00:00:00' # input('Input end time in HH:mm:ss format: ')

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'

### Determine how many granules exist over this time and area of interest.

In [5]:
# Create CMR parameters used for granule search. Modify params depending on bounding_box or polygon input.

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
aoi='1'
if aoi == '1':
# bounding box input:
    search_params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    # 'bounding_box': bounding_box
    }

granules = []
headers={'Accept': 'application/json'}
while True:
    response = requests.get(granule_search_url, params=search_params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    search_params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')

granule_sizes = [float(granule['granule_size']) for granule in granules]
print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')


There are 1474 granules of SPL4SMGP version 007 over my area and time of interest.
The average size of each granule is 143.54 MB and the total size of all 1474 granules is 211577.59 MB


Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

### Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request. 

### Query the service capability endpoint to gather service information needed below

In [6]:
# Query service capability URL 
from xml.etree import ElementTree as ET
capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

# Create session to store cookie and pass credentials to capabilities url
session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid, pswd))
print(response)
root = ET.fromstring(response.content)

#collect lists with each service option
subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]
if len(subagent) > 0 :

    # variable subsetting
    variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
    variables_raw = [variables[i]['value'] for i in range(len(variables))]
    variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
    variable_vals = [v.replace(':', '/') for v in variables_join]

    # reformatting
    formats = [Format.attrib for Format in root.iter('Format')]
    format_vals = [formats[i]['value'] for i in range(len(formats))]
    format_vals.remove('')

    # reprojection options
    projections = [Projection.attrib for Projection in root.iter('Projection')]

<Response [200]>


### Select subsetting, reformatting, and reprojection service options, if available.

In [7]:
#print service information depending on service availability and select service options
    
if len(subagent) < 1 :
    print('No services exist for', short_name, 'version', latest_version)
    agent = 'NO'
    bbox = ''
    time_var = ''
    reformat = ''
    projection = ''
    projection_parameters = ''
    coverage = ''
    Boundingshape = ''
else:
    # These reformatting options are available: ['HDF-EOS5', 'NetCDF4-CF', 'ASCII', 'KML', 'GeoTIFF']
    # These reprojection options are available with your requested format: ['GEOGRAPHIC', 'UNIVERSAL TRANSVERSE MERCATOR', 'CYLINDRICAL EQUAL AREA', 'NORTH POLAR STEREOGRAPHIC', 'SOUTH POLAR STEREOGRAPHIC']
    agent = ''
    subdict = subagent[0]
    if subdict['spatialSubsetting'] == 'true' and aoi == '1':
        Boundingshape = ''
        ss = 'n' # input('Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n)')
        if ss == 'y': bbox = bounding_box
        else: bbox = '' 
    if subdict['spatialSubsettingShapefile'] == 'true' and aoi == '2':
        bbox = ''
        ps = 'n' # input('Subsetting by geospatial file (Esri Shapefile, KML, etc.) is available. Would you like to request this service? (y/n)')
        if ps == 'y': Boundingshape = geojson
        else: Boundingshape = '' 
    if subdict['temporalSubsetting'] == 'true':
        ts = 'y'
        if ts == 'y': time_var = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time 
        else: time_var = ''
    else: time_var = ''
    if len(format_vals) > 0 :
        print('These reformatting options are available:', format_vals)
        reformat = 'NetCDF4-CF'
        if reformat == 'n': reformat = '' # Catch user input of 'n' instead of leaving blank
    else: 
        reformat = ''
        projection = ''
        projection_parameters = ''
    if len(projections) > 0:
        valid_proj = [] # select reprojection options based on reformatting selection
        for i in range(len(projections)):
            if 'excludeFormat' in projections[i]:
                exclformats_str = projections[i]['excludeFormat'] 
                exclformats_list = exclformats_str.split(',')
            if ('excludeFormat' not in projections[i] or reformat not in exclformats_list) and projections[i]['value'] != 'NO_CHANGE': valid_proj.append(projections[i]['value'])
        if len(valid_proj) > 0:
            print('These reprojection options are available with your requested format:', valid_proj)
            projection = 'GEOGRAPHIC' #input('If you would like to reproject, copy and paste the reprojection option you would like (make sure to omit quotes), otherwise leave blank.')
            # Enter required parameters for UTM North and South
            if projection == 'UTM NORTHERN HEMISPHERE' or projection == 'UTM SOUTHERN HEMISPHERE': 
                NZone = input('Please enter a UTM zone (1 to 60 for Northern Hemisphere; -60 to -1 for Southern Hemisphere):')
                projection_parameters = str('NZone:' + NZone)
            else: projection_parameters = ''
        else: 
            print('No reprojection options are supported with your requested format')
            projection = ''
            projection_parameters = ''
    else:
        print('No reprojection options are supported with your requested format')
        projection = ''
        projection_parameters = ''



These reformatting options are available: ['HDF-EOS5', 'NetCDF4-CF', 'ASCII', 'KML', 'GeoTIFF']
These reprojection options are available with your requested format: ['GEOGRAPHIC', 'UNIVERSAL TRANSVERSE MERCATOR', 'CYLINDRICAL EQUAL AREA', 'NORTH POLAR STEREOGRAPHIC', 'SOUTH POLAR STEREOGRAPHIC']


In [8]:
# # Select variable subsetting
# print(*variable_vals, sep = "\n") 
if len(subagent) > 0 :
    if len(variable_vals) > 0:
        v = 'y'
        if v == 'y':
            print('The', short_name, 'variables to select from include:')
            
            coverage = '/Geophysical_Data/precipitation_total_surface_flux,/cell_column,/cell_row,/cell_lat,/cell_lon'
        else: coverage = ''

#no services selected
if reformat == '' and projection == '' and projection_parameters == '' and coverage == '' and time_var == '' and bbox == '' and Boundingshape == '':
    agent = 'NO'

The SPL4SMGP variables to select from include:


### Select data access configurations

The data request can be accessed asynchronously or synchronously. The asynchronous option will allow concurrent requests to be queued and processed without the need for a continuous connection. Those requested orders will be delivered to the specified email address, or they can be accessed programmatically as shown below. Synchronous requests will automatically download the data as soon as processing is complete. The granule limits differ between these two options:

Maximum granules per synchronous request = 100 

Maximum granules per asynchronous request = 2000 

We will set the access configuration depending on the number of granules requested. For requests over 2000 granules, we will produce multiple API endpoints for each 2000-granule order. Please note that synchronous requests may take a long time to complete depending on request parameters, so the number of granules may need to be adjusted if you are experiencing performance issues. The `page_size` parameter can be used to adjust this number. 

In [9]:
#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

#Set the request mode to asynchronous if the number of granules is over 100, otherwise synchronous is enabled by default
if len(granules) > 100:
    request_mode = 'async'
    page_size = 2000
else: 
    page_size = 100
    request_mode = 'stream'

#Determine number of orders needed for requests over 2000 granules. 
page_num = math.ceil(len(granules)/page_size)

print('There will be', page_num, 'total order(s) processed for our', short_name, 'request.')

There will be 1 total order(s) processed for our SPL4SMGP request.


### Create the API endpoint 

Programmatic API requests are formatted as HTTPS URLs that contain key-value-pairs specifying the service operations that we specified above. The following command can be executed via command line, a web browser, or in Python below. 

In [10]:
if aoi == '1':
# bounding box search and subset:
    param_dict = {'short_name': short_name, 
                  'version': latest_version, 
                  'temporal': temporal, 
                  'time': time_var, 
                  # 'bounding_box': bounding_box, 
                  # 'bbox': bbox, 
                  'format': reformat, 
                  'projection': projection, 
                  'projection_parameters': projection_parameters, 
                  'Coverage': coverage, 
                  'page_size': page_size, 
                  'request_mode': request_mode, 
                  'agent': agent, 
                  'email': email, }

#Remove blank key-value-pairs
param_dict = {k: v for k, v in param_dict.items() if v != ''}

#Convert to string
param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items())
param_string = param_string.replace("'","")

#Print API base URL + request parameters
endpoint_list = [] 
for i in range(page_num):
    page_val = i + 1
    API_request = api_request = f'{base_url}?{param_string}&page_num={page_val}'
    endpoint_list.append(API_request)

print(*endpoint_list, sep = "\n") 

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=SPL4SMGP&version=007&temporal=2018-07-01T00:00:00Z,2019-01-01T00:00:00Z&format=NetCDF4-CF&projection=GEOGRAPHIC&Coverage=/Geophysical_Data/precipitation_total_surface_flux,/cell_column,/cell_row,/cell_lat,/cell_lon&page_size=2000&request_mode=async&email=raraki8159@sdsu.edu&page_num=1


### Request data

We will now download data using the Python requests library. The data will be downloaded directly to this notebook directory in a new Outputs folder. The progress of each order will be reported.

In [11]:
# Create an output folder if the folder does not already exist.
path = os.path.join(out_path, short_name + 'v1')
if not os.path.exists(path):
    os.mkdir(path)

# Different access methods depending on request mode:

if request_mode=='async':
    # Request data service for each page number, and unzip outputs
    for i in range(page_num):
        page_val = i + 1
        print('Order: ', page_val)

    # For all requests other than spatial file upload, use get function
        request = session.get(base_url, params=param_dict)

        print('Request HTTP response: ', request.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request.raise_for_status()
        print('Order request URL: ', request.url)
        esir_root = ET.fromstring(request.content)
        # print('Order request response XML content: ', request.content)

    #Look up order ID
        orderlist = []   
        for order in esir_root.findall("./order/"):
            orderlist.append(order.text)
        orderID = orderlist[0]
        print('order ID: ', orderID)

    #Create status URL
        statusURL = base_url + '/' + orderID
        print('status URL: ', statusURL)

    #Find order status
        request_response = session.get(statusURL)    
        print('HTTP response from order response URL: ', request_response.status_code)

    # Raise bad request: Loop will stop for bad response code.
        request_response.raise_for_status()
        request_root = ET.fromstring(request_response.content)
        statuslist = []
        for status in request_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Data request ', page_val, ' is submitting...')
        print('Initial request status is ', status)

    #Continue loop while request is still processing
        while status == 'pending' or status == 'processing': 
            # print('Status is not complete. Trying again.')
            time.sleep(60*5)
            loop_response = session.get(statusURL)

    # Raise bad request: Loop will stop for bad response code.
            loop_response.raise_for_status()
            loop_root = ET.fromstring(loop_response.content)

    #find status
            statuslist = []
            for status in loop_root.findall("./requestStatus/"):
                statuslist.append(status.text)
            status = statuslist[0]
            # print('Retry request status is: ', status)
            if status == 'pending' or status == 'processing':
                continue

    #Order can either complete, complete_with_errors, or fail:
    # Provide complete_with_errors error message:
        if status == 'complete_with_errors' or status == 'failed':
            messagelist = []
            for message in loop_root.findall("./processInfo/"):
                messagelist.append(message.text)
            print('error messages:')
            pprint.pprint(messagelist)

    # Download zipped order if status is complete or complete_with_errors
        if status == 'complete' or status == 'complete_with_errors':
            for nzip in range(1,5):
                try:
                    downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/'  + orderID + '.zip?' +  str(nzip)
                    print('Zip download URL: ', downloadURL)
                    print('Beginning download of zipped output...')
                    zip_response = session.get(downloadURL)
                    # Raise bad request: Loop will stop for bad response code.
                    zip_response.raise_for_status()
                    with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                        z.extractall(path)
                except:
                    continue
            print('Data request', page_val, 'is complete.')
        else: print('Request failed.')
            
else:
    for i in range(page_num):
        page_val = i + 1
        print('Order: ', page_val)
        print('Requesting...')
        request = session.get(base_url, params=param_dict)
        print('HTTP response from order response URL: ', request.status_code)
        request.raise_for_status()
        d = request.headers['content-disposition']
        fname = re.findall('filename=(.+)', d)
        dirname = os.path.join(path,fname[0].strip('\"'))
        print('Downloading...')
        open(dirname, 'wb').write(request.content)
        print('Data request', page_val, 'is complete.')
    
    # Unzip outputs
    for z in os.listdir(path): 
        if z.endswith('.zip'): 
            zip_name = path + "/" + z 
            zip_ref = zipfile.ZipFile(zip_name) 
            zip_ref.extractall(path) 
            zip_ref.close() 
            os.remove(zip_name) 


Order:  1
Request HTTP response:  201
Order request URL:  https://n5eil02u.ecs.nsidc.org/egi/request?short_name=SPL4SMGP&version=007&temporal=2018-07-01T00%3A00%3A00Z%2C2019-01-01T00%3A00%3A00Z&format=NetCDF4-CF&projection=GEOGRAPHIC&Coverage=%2FGeophysical_Data%2Fprecipitation_total_surface_flux%2C%2Fcell_column%2C%2Fcell_row%2C%2Fcell_lat%2C%2Fcell_lon&page_size=2000&request_mode=async&email=raraki8159%40sdsu.edu
order ID:  5000004009628
status URL:  https://n5eil02u.ecs.nsidc.org/egi/request/5000004009628
HTTP response from order response URL:  201
Data request  1  is submitting...
Initial request status is  pending
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000004009628.zip?1
Beginning download of zipped output...
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000004009628.zip?2
Beginning download of zipped output...
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000004009628.zip?3
Beginning download of zipped output...
Zip download URL:  https://n5ei

In [73]:
# https://n5eil02u.ecs.nsidc.org/esir/<order ID>.html/
# 2016-2017
# 5000003983740
# 5000003983761
# 5000003983789
# 5000003983808
# 5000003983849
# 5000003983877
# 5000003983895
# 5000003983919
# 5000003983937


# 2017-2018
# 5000003985742
# 5000003985793
# 5000003985831
# 5000003985901
# 5000003985864
# 5000003985933
# 5000003985965
# 5000003986003



### Finally, we will clean up the Output folder by removing individual order folders:

In [99]:
# Unzip outputs

for z in os.listdir(path): 
    if z.endswith('.zip'): 
        zip_name = path + "/" + z 
        zip_ref = zipfile.ZipFile(zip_name) 
        zip_ref.extractall(path) 
        zip_ref.close() 
        os.remove(zip_name) 

In [100]:
# Clean up Outputs folder by removing individual granule folders 
path = r'C:\Users\raraki8159\Downloads'
for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
    for name in dirs:
       if os.path.isdir(os.path.join(root, name)):
            try:
                os.rmdir(os.path.join(root, name))    
            except OSError:
                pass

### To review, we have explored data availability and volume over a region and time of interest, discovered and selected data customization options, constructed an API endpoint for our request, and downloaded data directly to our local machine. You are welcome to request different data sets, areas of interest, and/or customization services by re-running the notebook or starting again at the 'Select a data set of interest' step above. 

In [3]:
import os
import datetime

def is_complete(folder):
    # Create a dictionary to store the number of files seen for each date
    seen_dates = {}
    
    # Loop through the files in the folder
    for file in os.listdir(folder):
        # Check if the file name matches the pattern
        # Vv7032 is v7
        # Vv7030 is v6
        if not file.startswith("SMAP_L4_SM_gph_") or not file.endswith("Vv7032_001_HEGOUT.nc"):
            continue
        
        # Extract the date from the file name
        date_str = file[15:23]
        # print(date_str)
        # print(date_str)
        try:
            date = datetime.datetime.strptime(date_str, "%Y%m%d").date()
        except ValueError:
            continue

        # Add the date to the dictionary of seen dates only if it falls within the specified range
        if date >= datetime.date(2015, 3, 31) and date <= datetime.date(2022, 3, 30):
            if date in seen_dates:
                seen_dates[date] += 1
            else:
                seen_dates[date] = 1
        
    
    # Create a set of all the dates from March 31, 2015 to March 30, 2022
    all_dates = set()
    start_date = datetime.date(2015, 3, 31)
    end_date = datetime.date(2022, 3, 30)
    current_date = start_date
    while current_date <= end_date:
        all_dates.add(current_date)
        current_date += datetime.timedelta(days=1)
        
    # Check if there are 8 files per day
    missing_dates = set()
    for date in all_dates:
        if date not in seen_dates or seen_dates[date] != 8:
            missing_dates.add(date)

    if missing_dates:
        print("Missing dates:")
        for date in sorted(missing_dates):
            print(f"{date} - {8 - seen_dates.get(date,0)} files missing")
    else:
        return True
    return False

folder = r'G:\Araki\SMSigxSMAP\1_data\SPL4SMGP'
if is_complete(folder):
    print("Daily data is complete from March 31, 2015 to March 30, 2022 with 8 files per day.")
else:
    print("Daily data is not complete from March 31, 2015 to March 30, 2022 with 8 files per day.")


Missing dates:
2021-10-01 - 8 files missing
2021-10-02 - 8 files missing
2021-10-03 - 8 files missing
2021-10-04 - 8 files missing
2021-10-05 - 8 files missing
2021-10-06 - 8 files missing
2021-10-07 - 8 files missing
2021-10-08 - 8 files missing
2021-10-09 - 8 files missing
2021-10-10 - 8 files missing
2021-10-11 - 8 files missing
2021-10-12 - 8 files missing
2021-10-13 - 8 files missing
2021-10-14 - 8 files missing
2021-10-15 - 8 files missing
2021-10-16 - 8 files missing
2021-10-17 - 8 files missing
2021-10-18 - 8 files missing
2021-10-19 - 8 files missing
2021-10-20 - 8 files missing
2021-10-21 - 8 files missing
2021-10-22 - 8 files missing
2021-10-23 - 8 files missing
2021-10-24 - 8 files missing
2021-10-25 - 8 files missing
2021-10-26 - 8 files missing
2021-10-27 - 8 files missing
2021-10-28 - 8 files missing
2021-10-29 - 8 files missing
2021-10-30 - 8 files missing
2021-10-31 - 8 files missing
2021-11-01 - 8 files missing
2021-11-02 - 8 files missing
2021-11-03 - 8 files missing